In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import random
import matplotlib.pyplot as plt

COLOR_TO_CLASS = {
    (230, 25, 75): 0,
    (145, 30, 180): 1,
    (60, 180, 75): 2,
    (245, 130, 48): 3,
    (255, 255, 255): 4,
    (0, 130, 200): 5,
    (255, 0, 255): 6
}

CLASS_TO_COLOR = {
    0: (230, 25, 75),
    1: (145, 30, 180),
    2: (60, 180, 75),
    3: (245, 130, 48),
    4: (255, 255, 255),
    5: (0, 130, 200),
}

IGNORE_COLOR = (255, 0, 255)
NUM_CLASSES = 6
CLASS_NAMES = ['Building', 'Clutter', 'Vegetation', 'Water', 'Background', 'Car']


def decode_coloured_label(label_rgb):
    label_rgb = tf.cast(label_rgb, tf.uint8)
    flat = tf.reshape(label_rgb, [-1, 3])
    keys = tf.constant(list(COLOR_TO_CLASS.keys()), dtype=tf.uint8)
    values = tf.constant(list(COLOR_TO_CLASS.values()), dtype=tf.int32)
    match = tf.reduce_all(tf.equal(tf.expand_dims(flat, 1), keys), axis=2)
    indices = tf.argmax(tf.cast(match, tf.int32), axis=1)
    return tf.reshape(indices, tf.shape(label_rgb)[:2])

def _load_npy(path):
    return np.load(path.decode("utf-8")).astype(np.float32)

def load_image_paths(df, image_dir, elevation_dir, slope_dir, label_dir):
    tile_ids = df["tile_id"].tolist()
    image_paths = [os.path.join(image_dir, f"{tid}-ortho.png") for tid in tile_ids]
    elevation_paths = [os.path.join(elevation_dir, f"{tid}-elev.npy") for tid in tile_ids]
    slope_paths = [os.path.join(slope_dir, f"{tid}-slope.npy") for tid in tile_ids]
    label_paths = [os.path.join(label_dir, f"{tid}-label.png") for tid in tile_ids]
    return image_paths, elevation_paths, slope_paths, label_paths, tile_ids

def augment_image(rgb, elev, slope, label):
    """
    Applies random flips, rotation, and contrast.
    Ensures 3D shape for all inputs.
    """
    # Ensure tensors have shape [H, W, C]
    if tf.rank(elev) == 2:
        elev = tf.expand_dims(elev, axis=-1)
    if tf.rank(slope) == 2:
        slope = tf.expand_dims(slope, axis=-1)
    if tf.rank(label) == 2:
        label = tf.expand_dims(label, axis=-1)

    # --- Random Flips ---
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_left_right(rgb)
        elev = tf.image.flip_left_right(elev)
        slope = tf.image.flip_left_right(slope)
        label = tf.image.flip_left_right(label)

    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_up_down(rgb)
        elev = tf.image.flip_up_down(elev)
        slope = tf.image.flip_up_down(slope)
        label = tf.image.flip_up_down(label)

    # --- Random 90-degree rotation ---
    if tf.random.uniform([]) > 0.5:
        k = tf.random.uniform([], minval=1, maxval=4, dtype=tf.int32)
        rgb = tf.image.rot90(rgb, k=k)
        elev = tf.image.rot90(elev, k=k)
        slope = tf.image.rot90(slope, k=k)
        label = tf.image.rot90(label, k=k)

    # --- Random contrast (only RGB) ---
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.random_contrast(rgb, lower=0.8, upper=1.2)

    # Final shape correction
    label = tf.squeeze(label, axis=-1)  # Return label as [H, W]
    return rgb, elev, slope, label


def parse_elevation(rgb_path, elev_path, slope_path, label_path, tile_id,
                    split='train', augment=False, tile_size=256, dummy=False):
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    if dummy:
        elev = tf.random.uniform(tf.shape(rgb)[..., :1])
        slope = tf.random.uniform(tf.shape(rgb)[..., :1])
    else:
        elev = tf.numpy_function(_load_npy, [elev_path], tf.float32)
        elev = tf.expand_dims(elev, axis=-1)
        elev.set_shape([None, None, 1])

        slope = tf.numpy_function(_load_npy, [slope_path], tf.float32)
        slope = tf.expand_dims(slope, axis=-1)
        slope.set_shape([None, None, 1])

    if split == 'train' and augment:
        rgb, elev, slope, label = augment_image(rgb, elev, slope, label)

    rgb = tf.image.resize(rgb, [tile_size, tile_size])
    elev = tf.image.resize(elev, [tile_size, tile_size])
    slope = tf.image.resize(slope, [tile_size, tile_size])
    input_image = tf.concat([rgb, elev, slope], axis=-1)

    label.set_shape([None, None])
    label = tf.image.resize(label[..., tf.newaxis], [tile_size, tile_size], method='nearest')
    label = tf.squeeze(label, axis=-1)
    label = tf.cast(label, tf.int32)
    label_onehot = tf.one_hot(label, depth=NUM_CLASSES)

    return input_image, label_onehot

def parse_tile(rgb_path, label_path, tile_id, split='train', augment=False, tile_size=256):
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    if split == 'train' and augment:
        dummy = tf.zeros_like(rgb[..., :1])
        rgb, _, _, label = augment_image(rgb, dummy, dummy, label)

    rgb = tf.image.resize(rgb, [tile_size, tile_size])
    label = tf.image.resize(label[..., tf.newaxis], [tile_size, tile_size], method='nearest')
    label = tf.squeeze(label, axis=-1)
    label = tf.cast(label, tf.int32)
    label_onehot = tf.one_hot(label, depth=NUM_CLASSES)

    return rgb, label_onehot

def build_tf_dataset(df, image_dir, elev_dir, slope_dir, label_dir,
                     input_type='rgb', batch_size=32, split='train',
                     augment=False, shuffle=True, tile_size=256, dummy=False):
    
    image_paths, elev_paths, slope_paths, label_paths, tile_ids = load_image_paths(df, image_dir, elev_dir, slope_dir, label_dir)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, elev_paths, slope_paths, label_paths, tile_ids))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths), reshuffle_each_iteration=(split == 'train'))

    def map_fn(rgb_path, elev_path, slope_path, label_path, tile_id):
        if input_type == 'rgb':
            return parse_tile(rgb_path, label_path, tile_id, split, augment, tile_size)
        elif input_type == 'rgb_elev':
            return parse_elevation(rgb_path, elev_path, slope_path, label_path, tile_id, split, augment, tile_size, dummy=dummy)
        else:
            raise ValueError(f"Unsupported input_type: {input_type}")

    dataset = dataset.map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset